<a href="https://colab.research.google.com/github/Jozdien/reward-side-channels/blob/main/Reward_Side_Channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 0s (1,264 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...

In [3]:

# Special gym environment
!pip install gym[atari]

# For rendering environment, you can use pyvirtualdisplay.
!pip install pyvirtualdisplay
!pip install piglet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 2.9 MB/s 


In [4]:
#Install spinningup on CoLab
!git clone https://github.com/AmritaANair/spinningup-logchanges.git
#!pip install -e . # this will incur error: File "setup.py" not found. Directory cannot be installed in editable mode: /content
!pip install -e spinningup-logchanges

Cloning into 'spinningup-logchanges'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 223 (delta 25), reused 223 (delta 25), pack-reused 0
Receiving objects: 100% (223/223), 23.77 MiB | 39.97 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/spinningup-logchanges
     |████████████████████████████████| 1.6 MB 7.6 MB/s 
     |████████████████████████████████| 13.1 MB 6.5 MB/s 
     |████████████████████████████████| 2.5 MB 35.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 178 kB 53.6 MB/s 
     |████████████████████████████████| 110.5 MB 1.3 kB/s 
     |████████████████████████████████| 734.6 MB 18 kB/s 
     |███████████████████████

In [1]:

# To activate virtual display 
# need to run a script once for training an agent as follows
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

#
# Import libraries
#
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [5]:
from spinup import ppo_pytorch as ppo
import torch
import gym
import os.path as osp, time, atexit, os

env_fn = lambda : gym.make('LunarLander-v2')

ac_kwargs = dict(hidden_sizes=[64,64], activation=torch.nn.ReLU)

logger_kwargs = dict(output_dir='gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output', exp_name='experiment_name')

EPOCH = 20
LOSS = 0.4

In [7]:
ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=500, epochs=20, logger_kwargs=logger_kwargs)  #changed from steps_per_epoch=5000, epochs=250

Logging data to gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"ReLU",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.2,
    "env_fn":	"<function <lambda> at 0x7ffa7183ee60>",
    "epochs":	20,
    "exp_name":	"experiment_name",
    "gamma":	0.99,
    "lam":	0.97,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7ffa6c9d4a50>":	{
            "epoch_dict":	{},
            "exp_name":	"experiment_name",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output",
            "output_file":	{
                "<_io.TextIOWrapper name='gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"


In [9]:
import shutil

original = r'gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output'
target = r'gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/%i'%int(time.time())

shutil.copytree(original, target)

'gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/1658608763'

In [11]:
from spinup.utils.test_policy import load_policy_and_env as load_policy, run_policy
# Show policy
_, get_action = load_policy('gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output')
env2 = gym.make('LunarLander-v2')
env3 = wrap_env(env2)
run_policy(env3, get_action, max_ep_len=500, num_episodes=10)
env3.close()
show_video() 



Loading from gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output/pyt_save/model.pt.


Logging data to /content/gdrive/MyDrive/Reward Side Channels - Model Data/tmp/experiments/1658608778/progress.txt
Episode 0 	 EpRet -61.834 	 EpLen 164
Episode 1 	 EpRet -242.339 	 EpLen 225
Episode 2 	 EpRet -40.873 	 EpLen 118
Episode 3 	 EpRet 0.547 	 EpLen 145
Episode 4 	 EpRet -40.367 	 EpLen 106
Episode 5 	 EpRet -65.424 	 EpLen 175
Episode 6 	 EpRet -94.722 	 EpLen 70
Episode 7 	 EpRet -119.362 	 EpLen 232
Episode 8 	 EpRet -80.493 	 EpLen 121
Episode 9 	 EpRet -101.583 	 EpLen 235
-------------------------------------
|    AverageEpRet |           -84.6 |
|        StdEpRet |            62.1 |
|        MaxEpRet |           0.547 |
|        MinEpRet |            -242 |
|           EpLen |             159 |
-------------------------------------
